ERROR: Invalid requirement: 'transformers,'


In [ ]:
import model_loader
import pipeline 
from PIL import Image 
from pathlib import Path 
from transformers import CLIPTokenizer 
import torch

device = 'cpu'

allow_cuda = False 
allow_mps = False 

if torch.cuda.is_available():
    device = 'cuda'
elif (torch.backends.mps.is_available()) and allow_mps:
    device = 'mps' #what's this? 
print(f"device: {device}")

tokenizer = CLIPTokenizer('../data/vocab.json', merges_file='../data/merges.txt')
model_file = "../data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, device=device)

## TEXT TO IMAGE

prompt = "A dog with sunglasses, wearing a comfy hat, Looking at camera, highly detailed, ultra sharp, cinamtic, 100mm lense, 4k resolution"
uncond_prompt = ""  #aka negative prompt what you want to work againsti n cfg 
do_cfg = True 
cfg_scale = 8 #1-> 14

#IMAGE TO IMAGE 
input_image = None 
image_path = './images/elvis.jpg'
input_image = Image.open(image_path)
strength = 0.9 #how much noise to initially add i.e 1 means pure noise

#SAMPLER 

sampler = 'ddpm'
num_inference_steps = 50 
seed = 42

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale, 
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=device,
    idle_device='cpu',
    tokenizer=tokenizer
)

Image.fromarray(output_image)
